In [28]:
!pip install pymongo
!pip install requests
!pip install bs4
!pip install selenium
!pip install webdriver_manager
!pip install vaderSentiment

  Obtaining dependency information for vaderSentiment from https://files.pythonhosted.org/packages/76/fc/310e16254683c1ed35eeb97386986d6c00bc29df17ce280aed64d55537e9/vaderSentiment-3.3.2-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/126.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/126.0 kB ? eta -:--:--
   --- ------------------------------------ 10.2/126.0 kB ? eta -:--:--
   --------- ----------------------------- 30.7/126.0 kB 262.6 kB/s eta 0:00:01
   ---------------------------- ---------- 92.2/126.0 kB 655.4 kB/s eta 0:00:01
   -------------------------------------- 126.0/126.0 kB 741.7 kB/s eta 0:00:00


In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import pprint
import random
from bs4 import BeautifulSoup
from pymongo import MongoClient, errors
import re 
import logging
from selenium.webdriver.support.ui import WebDriverWait
from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.triggers.cron import CronTrigger
from apscheduler.schedulers.background import BackgroundScheduler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split

## Partie 01: Scrapping des restaurants

### 1.1- Configuration selenium

In [4]:
# Configuration 
options = Options()
#options.add_argument('--headless')  
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


In [6]:
# Start Chrome with WebDriver Manager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

In [7]:
# Set up logging to file
logging.basicConfig(filename='scraping_errors.log', level=logging.ERROR,
                    format='%(asctime)s - %(levelname)s - %(message)s')


### 1.2-Recherche des restaurants

In [8]:
# Navigate to yelp
url = "https://www.yelp.fr/"
driver.get(url)
time.sleep(random.uniform(2, 5))

In [9]:
#Close the cookie module
try:
    # Wait until the button is clickable
    cookie_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "onetrust-reject-all-handler"))
    )
    cookie_button.click()
    print("Cookie button clicked!")
except Exception as e:
    print("Error clicking the cookie button:", e)

Cookie button clicked!


In [10]:
input_element = driver.find_element(By.ID, 'search_description')

# Clear any existing text in the input field (optional)
input_element.clear()

# Send the value 'restaurants' to the input field
input_element.send_keys('restaurants')

In [11]:
# Find the button by its aria-label attribute or other attributes
submit_button = driver.find_element(By.XPATH, '//button[@aria-label="Recherche"]')

# Click on the submit button
submit_button.click()

In [12]:
current_url = driver.current_url
print(f"Current URL: {current_url}")

Current URL: https://www.yelp.fr/search?find_desc=restaurants&find_loc=Paris


### 1.3-Scrapping des restaurants

In [13]:
# function to scrape comments for each restaurant
def scrape_restaurant_comments(link):
    comment_data = []
    retries = 3  # Set number of retries for loading comments
    success = False

    while retries > 0 and not success:
        try:
            # Open a new window for comments
            driver.execute_script("window.open('');")
            new_window = driver.window_handles[-1]
            driver.switch_to.window(new_window)

            # Navigate to the restaurant's comment page
            new_url = url + link
            print(f"Visiting: {new_url}")
            driver.get(new_url)

            time.sleep(5)  # Allow time for the page to load

            # Wait for reviews to load
            WebDriverWait(driver, 40).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'list__09f24__ynIEd'))
            )

            # Parse the page source for comments
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            li_elements = soup.find_all('li', class_='y-css-1sqelp2')
            for li in li_elements:
                name = li.find('a', class_='y-css-1ijjqcc').text.strip() if li.find('a', class_='y-css-1ijjqcc') else "N/A"
                location = li.find('span', class_='y-css-1wfz87z').text.strip() if li.find('span', class_='y-css-1wfz87z') else "N/A"
                star_review = li.find('div', class_='y-css-dnttlc')['aria-label'].split(' ')[0] if li.find('div', class_='y-css-dnttlc') else "N/A"
                date = li.find('span', class_='y-css-1d8mpv1').text.strip() if li.find('span', class_='y-css-1d8mpv1') else "N/A"
                comment = li.find('span', class_='raw__09f24__T4Ezm').text.strip() if li.find('span', class_='raw__09f24__T4Ezm') else "N/A"

                comment_data.append({
                    "name": name,
                    "location": location,
                    "star_review": star_review,
                    "date": date,
                    "comment": comment
                })

            success = True  # Mark success if no exceptions occurred

        except Exception as e:
            print(f"Failed to load comments for {link}: {e}")
            retries -= 1
            time.sleep(5)  # Wait before retrying

        finally:
            # Close the new window and switch back to the main window
            if len(driver.window_handles) > 1:
                driver.switch_to.window(new_window)
                driver.execute_script("window.close();")
                driver.switch_to.window(driver.window_handles[0])

    return comment_data

In [14]:
# function to scrap data related to each restaurant
def scrape_page_function(ul_element):
    restaurant_data_list = []
    try:
        ul_html = ul_element.get_attribute("outerHTML")
        soup = BeautifulSoup(ul_html, 'lxml')
        list_items = soup.find_all('li')  # Find all <li> elements

        for li in list_items:
            # Extract restaurant name
            name_tag = li.find('a', class_='y-css-1ijjqcc')
            name = name_tag.get_text(strip=True) if name_tag else None

            if not name:  # Skip if no name
                continue

            # Extract stars (ratings)
            stars_tag = li.find('span', class_='y-css-1ugd8yy')
            stars = stars_tag.get_text(strip=True) if stars_tag else "No stars found"

            # Extract tags
            tags = [tag.get_text(strip=True) for tag in li.find_all('span', class_='y-css-1h5rysu')]
            tags = ", ".join(tags)

            # Extract address and number of reviews
            span_tags = li.find_all('span', class_='y-css-1d8mpv1')
            address = "No address found"
            reviews = []

            for span in span_tags:
                text = span.get_text(strip=True)
                if "review" in text.lower():
                    text = re.sub(r'[()]', '', text)  # Remove parentheses
                    reviews.append(text)
                else:
                    address = text
            reviews = ", ".join(reviews)

            # Extract the link to the restaurant page
            link_tag = li.find('a', class_='y-css-1ijjqcc')
            link = link_tag['href'] if link_tag else None

            # Fetch comments for the specific restaurant link
            comments = scrape_restaurant_comments(link) if link else []
            if link:
                print(f"Scraped comments for restaurant: {name}")  # Print message when comments are scraped

            # Store the restaurant data with comments
            restaurant_data = {
                "name": name,
                "stars": stars,
                "tags": tags,
                "address": address,
                "reviews": reviews,
                "comments": comments  # Add comments directly
            }
            restaurant_data_list.append(restaurant_data)

    except Exception as e:
        print(f"Error scraping page: {str(e)}")

    return restaurant_data_list

In [15]:
# function that scrapes data from all pages
def scrape_all_pages(driver):
    all_restaurant_data = []
    current_page = 0  # Counter for pages processed

    while True:  # Infinite loop until no more pages
        try:
            ul_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'list__09f24__ynIEd'))
            )

            # Scrape basic restaurant data and inline comments
            restaurant_data = scrape_page_function(ul_element)
            all_restaurant_data.extend(restaurant_data)

            print(f"Processed page {current_page + 1}. Moving to the next page.")

            # Navigate to the next page
            try:
                next_page_link = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "a.next-link"))
                )
                next_page_link.click()
                time.sleep(random.uniform(5, 10))  # Longer random delay
                current_page += 1  # Increment page counter
            except Exception as e:
                print(f"No more pages or error navigating to the next page: {e}")
                break  # Exit the loop if no "Next Page" link is found

        except Exception as e:
            print(f"An error occurred while scraping: {e}")
            break  # Exit the loop if there's an error

    return all_restaurant_data


In [16]:
# data scrapping
restaurant_list = scrape_all_pages(driver)


Visiting: https://www.yelp.fr//biz/le-comptoir-de-la-gastronomie-paris?osq=restaurants
Scraped comments for restaurant: Le Comptoir de la Gastronomie
Visiting: https://www.yelp.fr//biz/bistro-des-augustins-paris?osq=restaurants
Scraped comments for restaurant: Bistro des Augustins
Visiting: https://www.yelp.fr//biz/brasserie-bellanger-paris?osq=restaurants
Scraped comments for restaurant: Brasserie Bellanger
Visiting: https://www.yelp.fr//biz/le-corner-saint-germain-paris?osq=restaurants
Scraped comments for restaurant: Le Corner Saint Germain
Visiting: https://www.yelp.fr//biz/caf%C3%A9-blanc-paris-7?osq=restaurants
Scraped comments for restaurant: Café Blanc
Visiting: https://www.yelp.fr//biz/l-avant-comptoir-paris-3?osq=restaurants
Scraped comments for restaurant: L’Avant Comptoir
Visiting: https://www.yelp.fr//biz/la-r%C3%A9galade-paris-3?osq=restaurants
Scraped comments for restaurant: La Régalade
Visiting: https://www.yelp.fr//biz/la-fontaine-de-mars-paris-2?osq=restaurants
Scrap

Scraped comments for restaurant: Restaurant Georgette
Visiting: https://www.yelp.fr//biz/shouk-paris?osq=restaurants
Scraped comments for restaurant: Shouk
Visiting: https://www.yelp.fr//biz/la-cr%C3%AAte-paris-2?osq=restaurants
Scraped comments for restaurant: La Crête
Visiting: https://www.yelp.fr//biz/chez-julien-paris-2?osq=restaurants
Scraped comments for restaurant: Chez Julien
Visiting: https://www.yelp.fr//biz/tamarind-paris?osq=restaurants
Scraped comments for restaurant: Tamarind
Visiting: https://www.yelp.fr//biz/tire-bouchon-paris?osq=restaurants
Scraped comments for restaurant: Tire Bouchon
Processed page 7. Moving to the next page.
Visiting: https://www.yelp.fr//biz/shakespeare-and-company-paris?osq=restaurants
Scraped comments for restaurant: Shakespeare and Company
Visiting: https://www.yelp.fr//biz/restaurant-georgette-paris?osq=restaurants
Scraped comments for restaurant: Restaurant Georgette
Visiting: https://www.yelp.fr//biz/chungking-express-paris?osq=restaurants
S

Scraped comments for restaurant: L’Annexe
Processed page 13. Moving to the next page.
Visiting: https://www.yelp.fr//biz/l-officine-du-louvre-paris?osq=restaurants
Scraped comments for restaurant: L’Officine du Louvre
Visiting: https://www.yelp.fr//biz/les-philosophes-paris?osq=restaurants
Scraped comments for restaurant: Les Philosophes
Visiting: https://www.yelp.fr//biz/le-bistro-du-p%C3%A9rigord-paris-3?osq=restaurants
Scraped comments for restaurant: Le Bistro du Périgord
Visiting: https://www.yelp.fr//biz/baguetts-caf%C3%A9-paris?osq=restaurants
Scraped comments for restaurant: Baguett’s Café
Visiting: https://www.yelp.fr//biz/le-prince-racine-paris?osq=restaurants
Scraped comments for restaurant: Le Prince Racine
Visiting: https://www.yelp.fr//biz/l-auberge-des-deux-ponts-paris-3?osq=restaurants
Scraped comments for restaurant: L’Auberge des Deux Ponts
Visiting: https://www.yelp.fr//biz/le-basilic-paris-2?osq=restaurants
Scraped comments for restaurant: Le Basilic
Visiting: https

Scraped comments for restaurant: Café Madame
Visiting: https://www.yelp.fr//biz/pompei-paris?osq=restaurants
Scraped comments for restaurant: Pompei
Visiting: https://www.yelp.fr//biz/le-comptoir-du-relais-paris-3?osq=restaurants
Scraped comments for restaurant: Le Comptoir du Relais
Visiting: https://www.yelp.fr//biz/l-orangerie-paris-3?osq=restaurants
Scraped comments for restaurant: L’Orangerie
Visiting: https://www.yelp.fr//biz/l-aligot-paris?osq=restaurants
Scraped comments for restaurant: L’Aligot
Processed page 20. Moving to the next page.
Visiting: https://www.yelp.fr//biz/l-orangerie-paris-3?osq=restaurants
Scraped comments for restaurant: L’Orangerie
Visiting: https://www.yelp.fr//biz/l-aligot-paris?osq=restaurants
Scraped comments for restaurant: L’Aligot
Visiting: https://www.yelp.fr//biz/le-52-paris?osq=restaurants
Scraped comments for restaurant: Le 52
Visiting: https://www.yelp.fr//biz/restaurant-bachaumont-paris-2?osq=restaurants
Scraped comments for restaurant: Restaur

In [17]:
df_restaurants = pd.DataFrame(restaurant_list)

df_restaurants.head(10)

,name,stars,tags,address,reviews,comments
0,Le Comptoir de la Gastronomie,4.5,Français,Châtelet/Les Halles,1.4k review,"[{'name': 'Diana M.', 'location': 'Kittsee, Au..."
1,Bistro des Augustins,4.4,"Bistrot, Bars à vins",Saint-Michel/Odéon,525 reviews,"[{'name': 'Lance B.', 'location': 'South Jorda..."
2,Brasserie Bellanger,4.8,Français,Gare du Nord/La Chapelle,41 reviews,"[{'name': 'Zaraid J.', 'location': 'Boston, Ét..."
3,Le Corner Saint Germain,4.6,Français,Saint-Michel/Odéon,31 reviews,"[{'name': 'Sofia J.', 'location': 'The Woodlan..."
4,Café Blanc,4.4,"Français, Cafés et thés, Petit déjeuner & brunch",Palais Royal/Musée du Louvre,70 reviews,"[{'name': 'Larry G.', 'location': 'Fort Worth,..."
5,L’Avant Comptoir,4.4,"Bars à tapas, Bars à vins",Saint-Michel/Odéon,684 reviews,"[{'name': 'Janessa B.', 'location': 'Milton, É..."
6,La Régalade,4.5,Français,Châtelet/Les Halles,107 reviews,"[{'name': 'Michelle T.', 'location': 'Manhatta..."
7,La Fontaine de Mars,4.4,Français,Tour Eiffel/Champ de Mars,364 reviews,"[{'name': 'Revanth R.', 'location': 'San Franc..."
8,La Jacobine,4.5,"Salons de thé, Français",Saint-Michel/Odéon,388 reviews,"[{'name': 'Joe L.', 'location': 'Vancouver, Ca..."
9,Le Bistrot des Vosges,4.3,Bistrot,Bastille,91 reviews,"[{'name': 'Janice B.', 'location': 'Rockville,..."


In [18]:
df_unique_restaurants = df_restaurants.drop_duplicates(subset=["name", "address"])
print(len(df_unique_restaurants))

219


In [19]:
driver.quit()

In [20]:
# Save the DataFrame to a CSV file
df_unique_restaurants.to_csv('restaurants_data.csv', index=False)

print("Data has been saved to 'restaurants_data.csv'")

Data has been saved to 'restaurants_data.csv'


## Partie 2: Stockage des données

### 2.1- Configuration de la base de données

In [2]:
from pymongo import MongoClient, errors

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["yelp_data"]
collection = db["restaurants"]

try:
    # Create a unique index on the name and address fields
    collection.create_index([("name", 1), ("address", 1)], unique=True)
    print("Index created successfully on 'name' and 'address'.")
except errors.OperationFailure as e:
    print(f"Index creation failed: {str(e)}")
except Exception as e:
    print(f"An unexpected error occurred: {str(e)}")


Index created successfully on 'name' and 'address'.


### 2.2- Stockage des données

In [22]:
def store_restaurants_data(restaurants_list, collection):
    for restaurant in restaurants_list:
        try:
            # Check if a restaurant with the same name and address already exists
            existing_restaurant = collection.find_one({
                "name": restaurant["name"],
                "address": restaurant["address"]
            })

            if existing_restaurant:
                # Update comments if the restaurant already exists
                print(f"Restaurant {restaurant['name']} at {restaurant['address']} already exists. Updating comments.")
                for comment in restaurant.get("comments", []):
                    if comment not in existing_restaurant.get("comments", []):
                        collection.update_one(
                            {"_id": existing_restaurant["_id"]},
                            {"$push": {"comments": comment}}
                        )
                        print(f"Added new comment for {restaurant['name']} at {restaurant['address']}.")
                continue  # Skip inserting duplicate restaurant data

            # Insert the restaurant into the collection with comments
            collection.insert_one(restaurant)
            print(f"Inserted: {restaurant['name']} at {restaurant['address']}")

        except Exception as e:
            print(f"An error occurred while inserting {restaurant['name']} at {restaurant['address']}: {str(e)}")

In [23]:
store_restaurants_data(restaurant_list,collection)

Inserted: Le Comptoir de la Gastronomie at Châtelet/Les Halles
Inserted: Bistro des Augustins at Saint-Michel/Odéon
Inserted: Brasserie Bellanger at Gare du Nord/La Chapelle
Inserted: Le Corner Saint Germain at Saint-Michel/Odéon
Inserted: Café Blanc at Palais Royal/Musée du Louvre
Inserted: L’Avant Comptoir at Saint-Michel/Odéon
Inserted: La Régalade at Châtelet/Les Halles
Inserted: La Fontaine de Mars at Tour Eiffel/Champ de Mars
Inserted: La Jacobine at Saint-Michel/Odéon
Inserted: Le Bistrot des Vosges at Bastille
Inserted: Chez Mademoiselle at 4ème
Inserted: Le Comptoir at Denfert-Rochereau
Inserted: Les Antiquaires at Musée d'Orsay
Inserted: Le Café at Etienne Marcel/Montorgueil
Inserted: Au Bourguignon du Marais at 4ème
Inserted: L’Escargot at Châtelet/Les Halles
Inserted: Sacré Frenchy ! at Beaubourg
Inserted: Sanukiya at Palais Royal/Musée du Louvre
Inserted: Le P’tit Bistro at Châtelet/Les Halles
Inserted: Bouillon Chartier at 9ème
Inserted: Heureux Comme Alexandre at Saint-M

Inserted: Loubnane at Notre Dame De Paris
Restaurant Café Saint-Régis at Île de la Cité/Île Saint Louis already exists. Updating comments.
Restaurant Loubnane at Notre Dame De Paris already exists. Updating comments.
Inserted: Le Voltaire at Musée d'Orsay
Inserted: Le Roi du Pot au Feu at Saint-Lazare/Grands Magasins
Inserted: Jacopo at Champs-Elysées
Inserted: Le Sanglier Bleu at Montmartre
Inserted: La Buvette at Chemin Vert/Richard Lenoir
Inserted: Aux Trois Mailletz at Saint-Michel/Odéon
Inserted: El Nopal at 10ème
Inserted: La Crêperie at Sorbonne/Panthéon
Inserted: Kodawari Ramen at Palais Royal/Musée du Louvre
Inserted: Buzkashi at Place de Clichy
Inserted: La Boucherie Café at Place de Clichy
Inserted: Huitrerie Régis at Saint-Germain-Des-Prés
Inserted: Enza & Famiglia Trattoria Pasta at Châtelet/Les Halles
Inserted: Le Mistral at Châtelet/Les Halles
Inserted: Express de Lyon at Bercy
Inserted: Ma Salle à Manger at 1er
Inserted: Café New York at 16ème
Inserted: L’Ardoise at Pla

## Partie 3: automatisation de scrapping

In [21]:
def scheduled_scraping():
    logging.info("Starting scheduled scraping...")

    try:
       
        # Start scraping
        all_restaurant_data = scrape_all_pages(driver)
        
        logging.info(f"Scraping completed, total {len(all_restaurant_data)} restaurants scraped.")
        
        # You can add code to store or process scraped data here
        store_restaurants_data(restaurant_list,collection)

    except Exception as e:
        logging.error(f"Error during scraper run: {str(e)}")

    finally:
        driver.quit()  # Ensure the driver is closed after the scraping session


In [22]:
# Set up scheduler
scheduler = BackgroundScheduler()

# Define the cron trigger for scheduling daily at 8:30 AM
cron_trigger = CronTrigger(hour=11, minute=57, second=0, day_of_week='*')  # Runs every day at 8:30 AM

# Add the scraping task to the scheduler
scheduler.add_job(scheduled_scraping, cron_trigger)

# Start the scheduler
scheduler.start()



## Partie4 : Analyse et prédictions

### 4.1 - Prédire la note moyenne (stars) d'un restaurant en fonction du nombre de commentaires (reviews) qu'il a reçus

In [9]:
# get data from db
restaurants = list(collection.find({}, {"_id": 0, "reviews": 1, "stars": 1}))
df = pd.DataFrame(restaurants)


In [10]:
def convert_reviews(review):
   
    match = re.match(r'(\d+(\.\d+)?)\s*(k|K|m|M)?\s*review', review.strip())
    if match:
       
        number = float(match.group(1))  
        suffix = match.group(3)  
        
        if suffix in ['k', 'K']:
            return number * 1000 
        elif suffix in ['m', 'M']:
            return number * 1000000  
        else:
            return number  
    return None 



In [11]:
# data cleaning and treatment
df['reviews'] = df['reviews'].apply(convert_reviews)  
df['stars'] = pd.to_numeric(df['stars'], errors='coerce') 

df = df.dropna()
df

,stars,reviews
0,4.5,1400.0
1,4.4,525.0
2,4.8,41.0
3,4.6,31.0
4,4.4,70.0
...,...,...
212,4.5,270.0
213,4.0,5.0
214,4.2,285.0
215,4.8,59.0


In [12]:
# data splitting
X = df[['reviews']]
y = df['stars']


In [13]:
# data modeling
model = LinearRegression()
model.fit(X, y)

# model eval
y_pred = model.predict(X)
mae = mean_absolute_error(y, y_pred)
print(f"Erreur absolue moyenne (MAE) : {mae}")

# test : restaurants with 30 comments
predicted_rating = model.predict(pd.DataFrame([[30]], columns=["reviews"]))

print(f"Prédiction de la note moyenne pour un restaurant avec 30 commentaires : {predicted_rating[0]}")



Erreur absolue moyenne (MAE) : 0.22941187216520945
Prédiction de la note moyenne pour un restaurant avec 30 commentaires : 4.443807662145708


Nous constatons que le modèle prédit assez bien les notes des restaurants en général au vu du faible montant de l'erreur absolue moyenne (MAE); 0.229. Par exemple, pour un restaurant avec 30 commentaires, nous prédisons une note moyenne est de 4.44, ce qui correspond à ce qu'on observe en réalité, les restaurant avec plus de commentaires ont tendance à avoir une note plus élevée.

### 4.2 - Prédiction de la note des restaurants en fonction du sentiment moyen des commentaires

In [36]:
restaurants = list(collection.find({}, {"_id": 0, "name": 1, "comments": 1, "stars": 1}))
df = pd.DataFrame(restaurants)

In [37]:
#analyzer init
analyzer = SentimentIntensityAnalyzer()

In [38]:
def analyze_sentiment(comment):
    sentiment_score = analyzer.polarity_scores(comment)
    return sentiment_score['compound']  #sentiment global

df['sentiment'] = df['comments'].apply(lambda x: [analyze_sentiment(c['comment']) for c in x] if isinstance(x, list) else [])

In [39]:
# Add the average sentiment score for each restaurant = average of sentiment scores from reviews
df['average_sentiment'] = df['sentiment'].apply(lambda x: np.mean(x) if isinstance(x, list) and len(x) > 0 else None)
df['average_sentiment'] = df['average_sentiment'].fillna(0)

# Split the data into training and testing sets
X = df[['average_sentiment']]  # Average sentiment as a feature
y = df['stars']  # Target variable: restaurant ratings

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [40]:
model = LinearRegression()
model.fit(X_train, y_train)

# Predictions of restaurant ratings
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print(f"Erreur absolue moyenne (MAE) : {mae}")

# Test: Predict the stars for a restaurant with an average_sentiment score of 0.8
new_sentiment = 0.8
predicted_rating = model.predict(pd.DataFrame([[new_sentiment]], columns=["average_sentiment"]))
print(f"Prédiction de la note pour un restaurant avec sentiment moyen de {new_sentiment} : {predicted_rating[0]}")


Erreur absolue moyenne (MAE) : 0.24718884627627516
Prédiction de la note pour un restaurant avec sentiment moyen de 0.8 : 4.4460155086237005


Nous constatons que le modèle prédit assez bien la note des restaurants en fonction des sentiments des clients, au vu du faible montant de l'MAE; proche de zéro. Par exemple, pour un restaurant evec une moyenne de 0.8 qui représente des commentaires positifs , la prédiction est de 4.44 étoiles ce qui correspond aux valeurs réelles. 